In [ ]:
import random
import os
from os import listdir

In [ ]:
hashtags = {
    "cat":[],
    "dog":[],
    "bird":[]
}

Note that i already scraped the images from Instagram, Filtered them and uploaded them to kaggle. <br>
https://www.kaggle.com/mahmoudnoor/high-resolution-catdogbird-image-dataset-13000 <br>
pathes in this notebook are prepared to run directly on kaggle.

In [ ]:
### get images for each hashtag ###
for hashtag in hashtags.keys():
    hashtags[hashtag] = listdir(f'/kaggle/input/high-resolution-catdogbird-image-dataset-13000/{hashtag}/{hashtag}')
    hashtags[hashtag] = hashtags[hashtag][:4014] # all calsses must be of equal length (4014 is the smallest class length of my dataset)
    random.shuffle(hashtags[hashtag]) # in case you want to shuffle images

## train-test preparation

In [ ]:
def prepareTrainTestFolders(test_size = 0.25): # used to split images to train and test folders
  
  os.system(f"mkdir /kaggle/working/dataset")
  os.system(f"mkdir /kaggle/working/dataset/train")
  os.system(f"mkdir /kaggle/working/dataset/test")  
  
  for hashtag in hashtags.keys():
    os.system(f"mkdir /kaggle/working/dataset/train/{hashtag}")
    os.system(f"mkdir /kaggle/working/dataset/test/{hashtag}")
    
    length = len(hashtags[hashtag])
    
    for index, i in enumerate(hashtags[hashtag]):
      if  index <= length * test_size: 
        os.system(f"cp /kaggle/input/high-resolution-catdogbird-image-dataset-13000/{hashtag}/{hashtag}/{i} /kaggle/working/dataset/test/{hashtag}/")
      else:
        os.system(f"cp /kaggle/input/high-resolution-catdogbird-image-dataset-13000/{hashtag}/{hashtag}/{i} /kaggle/working/dataset/train/{hashtag}/")
    
    
prepareTrainTestFolders()

In [ ]:
import tensorflow as tf
from tf.keras.preprocessing.image import ImageDataGenerator

# create a new generator
imagegen = ImageDataGenerator(rescale=1.0/255.0) # Normalization of pixels values (1.0/255.0)
# load train data
train = imagegen.flow_from_directory("/kaggle/working/dataset/train/", class_mode='sparse', shuffle=False, target_size=(500, 500))
# load val data
val = imagegen.flow_from_directory("/kaggle/working/dataset/test/", class_mode='sparse', shuffle=False, target_size=(500, 500))

In [ ]:
# train.batch_size = 512 # set batch size

## Model Creation

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
### Create Model ###

model = Sequential()
model.add(Conv2D(32,3,activation="relu", input_shape=(500,500,3))) ### 3 because color_mode is RGB (1 for grayscale)
model.add(MaxPool2D())

model.add(Conv2D(64, 3, activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(128, 3, activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(3, activation="softmax"))

### Training

In [ ]:
opt = Adam()
model.compile(optimizer = opt , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])
history = model.fit(train, epochs = 15, validation_data = val)

In [ ]:
model.summary()
model.save("model.h5")

## Evalutation

In [ ]:
import matplotlib.pyplot as plt


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(15) ### number of epochs  ###

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()